In [ ]:
# Import dependencies
import os
import json

In [ ]:
%pip install --quiet "evadb[document,notebook]"
import evadb
cursor = evadb.connect().cursor()

In [ ]:
# Set your OpenAI key as an environment variable
import os
#os.environ['OPENAI_API_KEY'] = ''
open_ai_key = os.environ.get("OPENAI_API_KEY", "")


In [ ]:
# set up the extract columns UDF available at functions/extract_columns.py
cursor.query("""CREATE FUNCTION IF NOT EXISTS ExtractColumn
            IMPL  '../evadb/functions/extract_column.py';
                """).execute()

In [ ]:
# # delete the table if it already exists
cursor.query("""DROP TABLE IF EXISTS InputUnstructured
                """).execute()

# create the table specifying the type of the prompt column
cursor.query("""CREATE TABLE IF NOT EXISTS InputUnstructured (
             input_rows TEXT)
                """).execute()

In [ ]:
input_rows_list = ["The touch screen on my tablet stopped working for no reason.",
                    "Why does my computer take so long to start up? It's been like this for weeks.",
                    "My phone battery dies too quickly. I just bought it!",
                    "My headphones won't connect to my phone anymore, even though they used to work just fine.",
                    "The software update completely messed up my computer. Now nothing works properly."]

for input_row in input_rows_list:
    cursor.query(f"""INSERT INTO InputUnstructured (input_rows) VALUES ("{input_row}")""").execute()


In [ ]:
table = cursor.query("SELECT * FROM InputUnstructured;").df()

In [ ]:
table = cursor.query(
        """SELECT ExtractColumn("Issue Component","The component that is causing the issue", "string less than 2 words", input_rows) FROM InputUnstructured;"""
    ).df()

for _, row in table.iterrows():
    print(row['response'])